In [1]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf 
from tensorflow.examples.tutorials.mnist import input_data

In [13]:
def xavier_init(fan_in,fan_out,constant=1):
    low=-constant*np.sqrt(6.0/(fan_in+fan_out))
    high=constant*np.sqrt(6.0/(fan_in+fan_out))
    return tf.random_uniform((fan_in,fan_out),minval=low,maxval=high,dtype=tf.float32)


In [19]:
class AdditiveGaussianNoiseAutoencoder(object):
    def __init__(self,n_input,n_hidden,transfer_function=tf.nn.softplus,
                optimizer=tf.train.AdamOptimizer(),scale=0.1):
        self.n_input=n_input
        self.n_hidden=n_hidden
        self.transfer=transfer_function
        self.scale=tf.placeholder(tf.float32)
        self.training_scale=scale
        network_weights=self._initialize_weights()
        self.weights=network_weights
        self.x=tf.placeholder(tf.float32,[None,n_input])
        self.hidden=self.transfer(tf.add(tf.matmul(
                            self.x+scale*tf.random_normal((n_input,)),
                                self.weights['w1']),self.weights['b1']))
        self.reconstruction=tf.add(tf.matmul(self.hidden,self.weights['w2']),self.weights['b2'])
        self.cost=0.5*tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction,self.x),2.0))
        self.optimizer=optimizer.minimize(self.cost)
        init=tf.global_variables_initializer()
        self.sess=tf.Session()
        self.sess.run(init)
    def _initialize_weights(self):
        all_weights=dict()
        all_weights['w1']=tf.Variable(xavier_init(self.n_input,
                                                self.n_hidden))
        all_weights['b1']=tf.Variable(tf.zeros([self.n_hidden],
                                              dtype=tf.float32))
        all_weights['w2']=tf.Variable(tf.zeros([self.n_hidden,self.n_input],dtype=tf.float32))
        all_weights['b2']=tf.Variable(tf.zeros([self.n_input],dtype=tf.float32))
        return all_weights
    
    def partial_fit(self,X):
        cost,opt=self.sess.run((self.cost,self.optimizer),
                              feed_dict={self.x:X,self.scale:self.training_scale})
        return cost
    
    def calc_total_cost(self,X):
        return self.sess.run(self.cost,
                             feed_dict={self.x:S,self.scale:self.training_scale})
    
    def transform(self,X):
        return self.sess.run(self.hidden,
                            feed_dict={
                                self.x:X,self.scale:self.training_scale
                            })
    def getWeights(self):
        return self.sess.run(self.weights['w1'])
    
    def getBiases(self):
        return self.sess.run(self.weights['b1'])
        
        

In [5]:
mnist=input_data.read_data_sets('MNIST_DATA',one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz


In [6]:
def standard_scale(X_train,X_test):
    preprocessor=prep.StandardScaler().fit(X_train)
    X_train=preprocessor.transform(X_train)
    X_test=preprocessor.transform(X_test)
    return X_train,X_test


In [24]:
def get_random_block_from_data(data,batch_size):
    start_index=np.random.randint(0,len(data)-batch_size)
    return data[start_index:(start_index+batch_size)]
X_train,X_test=standard_scale(mnist.train.images,mnist.test.images)

In [26]:
n_samples=int(mnist.train.num_examples)
training_epochs=200
batch_size=128
display_step=1

In [20]:
autoencoder=AdditiveGaussianNoiseAutoencoder(n_input=784,
                                            n_hidden=200,
                                            transfer_function=tf.nn.softplus,
                                            optimizer=tf.train.AdamOptimizer(learning_rate=0.001),
                                            scale=0.01)

In [27]:
for epoch in range(training_epochs):
    avg_cost=0
    total_batch=int(n_samples/batch_size)
    for i in range(total_batch):
        batch_xs=get_random_block_from_data(X_train,batch_size)
        cost=autoencoder.partial_fit(batch_xs)
        avg_cost+=cost/n_samples*batch_size
    if epoch%1==0:
        print("epoch:",'%04d'%(epoch+1),'cost=','{:.9f}'.format(avg_cost))

epoch: 0001 cost= 7788.887432955
epoch: 0002 cost= 8571.154782955
epoch: 0003 cost= 7711.643556818
epoch: 0004 cost= 8305.577836364
epoch: 0005 cost= 7587.538670455
epoch: 0006 cost= 7913.662339205
epoch: 0007 cost= 7634.894287500
epoch: 0008 cost= 7561.470842045
epoch: 0009 cost= 7751.009475000
epoch: 0010 cost= 8255.901509659
epoch: 0011 cost= 7756.619589205
epoch: 0012 cost= 7702.181657955
epoch: 0013 cost= 7673.500871591
epoch: 0014 cost= 7561.921728977
epoch: 0015 cost= 7854.827261932
epoch: 0016 cost= 7860.479686364
epoch: 0017 cost= 8335.805289205
epoch: 0018 cost= 8291.768567614
epoch: 0019 cost= 8187.180946591
epoch: 0020 cost= 6668.428705682
epoch: 0021 cost= 7254.599762500
epoch: 0022 cost= 7735.550426705
epoch: 0023 cost= 7372.236004545
epoch: 0024 cost= 7946.053638636
epoch: 0025 cost= 7800.935125000
epoch: 0026 cost= 7327.303769886
epoch: 0027 cost= 7114.259218182
epoch: 0028 cost= 7832.343958523
epoch: 0029 cost= 7367.082698864
epoch: 0030 cost= 7979.718875568
epoch: 003